In [3]:
from analysis.ipstartup import *
import aws
import fabric.api as fab
import fabric

# set verbosity=0
#fabric.state.output = {k:False for k,v in output.items()}


sm1 = aws.getRes("sm1", aws.ec2.instances)
fab.env.host_string = sm1.public_ip_address

[root:INFO]:starting (cellevents.py\31, time=17:29)


time: 1.67 s


In [23]:
import analysis.explore

ImportError: bad magic number in 'analysis': b'\x03\xf3\r\n'

In [22]:
""" explore tools excluding plots """

import os
import webbrowser
import logging as log
from IPython.display import display as d
from IPython.core.display import HTML
import pandas as pd

try:
    import xlwings as xw
except:
    pass

def htmlo(html, filename=None):
    """ shows html text in a browser """
    if not filename:    
        filename = "_temp.html"
    with open(filename, 'w', encoding="utf8") as f:
        f.write(html)
    webbrowser.open('file://' + os.path.realpath(filename))

def xlo(df, filename=None):
    """ show pandas dataframe or series in excel sheet
        uses xlwings which allows writing to open file
    """
    if not filename:    
        filename = "_temp.xlsx"
    if not os.path.isfile(filename):
        wb = xw.Book()
        xw.sheets("Sheet2").delete()
        xw.sheets("Sheet3").delete()
    else:
        wb = xw.Book(filename)
        xw.sheets.add()
    xw.sheets.active.range("A1").value = df
    wb.save(filename)

def stable(*args, **kwargs):
    """ survival table showing counts and %lived
        args[0] is row target died/lived
        args[1] is list of column variables
    """
    # enable assign to args
    args = list(args)
    # if args[1] is single column then make it a list
    if not isinstance(args[1], list):
        args[1] = [args[1]]
    
    title = ", ".join([col.name for col in args[1]])
    d(HTML("<h4>%s</h4>"%title))
    table = pd.crosstab(*args, **kwargs)
    table.index = ["died", "lived"]
    table.loc["rate"] = table.iloc[-1] / table.sum()
    ftable = table.style.format("{:.0%}", subset=pd.IndexSlice["rate",:])
    d(ftable)
    
def qcut(*args, **kwargs):
    """ override pd.qcut to adjust to "non-unique bin edges" exception 
    args[0] is variable
    args[1] is number of cuts
    """
    args = list(args)
    while True:
        try:
            return pd.qcut(*args, **kwargs)
        except ValueError:
            args[1] = args[1] - 1
            if args[1] < 1:
                raise "Cannot divide %s into %s percentiles or less"%(args[0].name, args[1])
            log.warning("reducing percentiles for %s to %s"%(args[0].name, args[1]))


In [11]:
# show servers
aws.show()

[root:INFO]:starting (cellevents.py\31, time=11:53)


,name,instance_id,image,type,state,ip
0,sm1,i-0fba41e915de786b3,ami-c51e3eb6,t2.micro,running,34.248.253.147


time: 480 ms


In [79]:
aws.install_projects()

[root:INFO]:starting (cellevents.py\31, time=16:18)


[34.248.253.147] run: mkdir basics || true
[34.248.253.147] out: mkdir: cannot create directory ‘basics’: File exists
[34.248.253.147] out: 

[localhost] local: git archive HEAD > basics.tar
[34.248.253.147] put: C:\Users\s\Documents\py\apps\aws\..\basics\basics.tar -> /home/ec2-user/basics.tar
[34.248.253.147] run: tar xf basics.tar -C basics
[34.248.253.147] run: rm basics.tar
[34.248.253.147] run: mkdir analysis || true
[34.248.253.147] out: mkdir: cannot create directory ‘analysis’: File exists
[34.248.253.147] out: 

[localhost] local: git archive HEAD > analysis.tar
[34.248.253.147] put: C:\Users\s\Documents\py\apps\aws\..\analysis\analysis.tar -> /home/ec2-user/analysis.tar
[34.248.253.147] run: tar xf analysis.tar -C analysis
[34.248.253.147] run: rm analysis.tar
[34.248.253.147] run: mkdir meetup || true
[34.248.253.147] out: mkdir: cannot create directory ‘meetup’: File exists
[34.248.253.147] out: 

[localhost] local: git archive HEAD > meetup.tar
[34.248.253.147] put: C:\Us

In [83]:
aws.restart_notebook()

[root:INFO]:starting (cellevents.py\31, time=16:23)


[34.248.253.147] run: docker rm -f notebook || true
[34.248.253.147] out: notebook
[34.248.253.147] out: 

[34.248.253.147] run: docker run -v $PWD/.ssh:/root/.ssh -v $PWD/.jupyter:/root/.jupyter -v $PWD:/host -w=/host -p 8888:8888 -d -i --name notebook kaggle/python
[34.248.253.147] out: 23201cb9fa25d661710d2ffdd84d42519abb332c7ce8bc2d59971b7a74d58671
[34.248.253.147] out: 

[34.248.253.147] run: docker exec notebook python basics/pathconfig.py
[34.248.253.147] out: [root:INFO]:starting (pathconfig.py\10, time=16:23)
[34.248.253.147] out: 

[34.248.253.147] run: docker exec -d notebook jupyter notebook
time: 4.55 s


In [84]:
aws.restart_meetup()

[root:INFO]:starting (cellevents.py\31, time=16:23)


[34.248.253.147] run: docker rm -f meetup || true
[34.248.253.147] out: meetup
[34.248.253.147] out: 

[34.248.253.147] run: docker run -v $PWD:/host -w=/host -d -i --name meetup kaggle/python
[34.248.253.147] out: 08ca6a160cac4f6e6ca5694c8758ad1ad2825f9f36731b0909c62562be58335d
[34.248.253.147] out: 

[34.248.253.147] run: docker exec meetup python basics/pathconfig.py
[34.248.253.147] out: [root:INFO]:starting (pathconfig.py\10, time=16:24)
[34.248.253.147] out: 

[34.248.253.147] run: docker exec -d meetup python meetup/meetup.py
time: 4.89 s


In [104]:
# check running
_ = fab.run("docker exec meetup ps -t | grep meetup")

[root:INFO]:starting (cellevents.py\31, time=17:10)


[34.248.253.147] run: docker exec meetup ps -t | grep meetup
[34.248.253.147] out:    10 ?        Ss     0:00 python meetup/meetup.py
[34.248.253.147] out: 

time: 374 ms
